In [ ]:
%run -n main.py

In [ ]:
# lines = read_lines(DOTENV_PATH)
# env = dict(parse_dotenv(lines))
# openai.api_key = env['OPENAI_TOKEN']

# tasks

In [ ]:
table = pd.read_csv('tasks/ru_user_oriented.csv')
table = table[~table.done.isnull()]
table = table.where(pd.notnull(table), None)
table = table[['id', 'instruction', 'input']]
ru_user_oriented_items = table.to_dict(orient='records')

# evals

## llama_7b_ru_turbo_alpaca_lora

In [ ]:
# LLM.int8() requires Turing or Ampere GPUs.
# WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.7/targets/x86_64-linux/lib'

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name = 'IlyaGusev/llama_7b_ru_turbo_alpaca_lora'

config = PeftConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,

    # Overriding torch_dtype=None with `torch_dtype=torch.float16
    torch_dtype=torch.float16,
    
    # A device map needs to be passed to run convert models into mixed-int8
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = PeftModel.from_pretrained(model, model_name)

In [ ]:
%run -n main.py
lines = read_lines('evals/gusev_7b_ru_alpaca_lora.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_ru_alcapa_prompt(item)
    output = ru_alpaca_complete(prompt, model, tokenizer)
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/gusev_7b_ru_alpaca_lora.jsonl', lines)

## llama_13b_ru_turbo_alpaca_lora

In [ ]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.7/targets/x86_64-linux/lib'

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name = 'IlyaGusev/llama_13b_ru_turbo_alpaca_lora'

config = PeftConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = PeftModel.from_pretrained(model, model_name)

In [ ]:
%run -n main.py
lines = read_lines('evals/gusev_13b_ru_alpaca_lora.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_ru_alcapa_prompt(item)
    output = ru_alpaca_complete(prompt, model, tokenizer)
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/gusev_13b_ru_alpaca_lora.jsonl', lines)

## instruct_rugptlarge

In [ ]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

model_name = 'AlexWortega/instruct_rugptlarge'

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
special_tokens_dict = {
    'additional_special_tokens': [
        '<code>', '</code>',
        '<instructionS>', '<instructionE>',
        '<next>'
    ]
}
tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
device = 'cuda:0'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
%run -n main.py
lines = read_lines('evals/wortega_instruct_rugpt_large.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_instruct_rugpt(item)
    output = instruct_rugpt_complete(prompt, model, tokenizer)
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/wortega_instruct_rugpt_large.jsonl', lines)

## gpt-3.5-turbo

In [ ]:
lines = read_lines('evals/openai_turbo.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_chat_complete(prompt, model='gpt-3.5-turbo')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_turbo.jsonl', lines)

## text-davinci-003

In [ ]:
lines = read_lines('evals/openai_davinci_003.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='text-davinci-003')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_003.jsonl', lines)

## text-davinci-002

In [ ]:
lines = read_lines('evals/openai_davinci_002.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='text-davinci-002')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_002.jsonl', lines)

## davinci-001

In [ ]:
lines = read_lines('evals/openai_davinci_001.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='davinci')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_001.jsonl', lines)